In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification

/Users/mahas/PycharmProjects/STIRS/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mahas/PycharmProjects/STIRS/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Tokenize and encode the data using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 128

In [3]:
DATASET_SIZE = 1000

import pandas as pd
df = pd.read_csv("../jigsaw/dataset_text_target.csv")
df = df.sample(frac=0.1).reset_index(drop=True)
df_true = df[df.target > 0.5][:DATASET_SIZE//2]
df_false = df[df.target <= 0.5][:DATASET_SIZE//2]
df = pd.concat([df_true, df_false], axis=0)
mapper = lambda x: 1 if x > 0.5 else 0
df.target = df.target.apply(mapper)
df = df.sample(frac=1).reset_index(drop=True)

In [13]:
df.target.value_counts()

target
0    500
1    500
Name: count, dtype: int64

In [4]:
bert_model = TFBertForSequenceClassification.from_pretrained('../bert_model_trained', num_labels=2)
from sklearn.metrics import classification_report
import tensorflow as tf
import time

2025-05-12 12:40:13.507734: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-05-12 12:40:13.507757: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-12 12:40:13.507761: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1747033813.507771  674167 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1747033813.507790  674167 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../bert_model_trained.
If your task is similar to the task the model of the chec

In [5]:
# encode once just in case theres any caching going on
X_test_encoded = tokenizer.batch_encode_plus(
    df.comment_text.tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_len,
    add_special_tokens=True,
    return_tensors='tf'
)



In [6]:
start = time.time()

X_test_encoded = tokenizer.batch_encode_plus(
    df.comment_text.tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_len,
    add_special_tokens=True,
    return_tensors='tf'
)


y_pred = bert_model.predict(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])
y_pred = tf.argmax(y_pred.logits, axis=1).numpy()

end = time.time()

2025-05-12 12:40:17.289251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 36s 1s/step


In [7]:
print(f"Time taken per message for BERT: {round(1000*(end-start) / len(df), 3)} ms")

Time taken per message for BERT: 36.459 ms


In [8]:
print(classification_report(y_pred, df.target))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92       448
           1       0.98      0.88      0.93       552

    accuracy                           0.92      1000
   macro avg       0.92      0.93      0.92      1000
weighted avg       0.93      0.92      0.92      1000



In [9]:
from keras.models import load_model
cnn_model = load_model("../cnn_model_trained/cnn_model_trained.keras")

/Users/mahas/PycharmProjects/STIRS/.venv/lib/python3.9/site-packages/keras/src/trainers/trainer.py:212: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False`.
  warnings.warn(


In [10]:
start = time.time()

X_test_encoded = tokenizer.batch_encode_plus(
    df.comment_text.tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_len,
    add_special_tokens=True,
    return_tensors='tf'
)

y_pred = cnn_model.predict(X_test_encoded["input_ids"])
y_pred = [1 if y > 0.5 else 0 for y in y_pred]

end = time.time()

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [11]:
print(f"Time taken per message for CNN: {round(1000*(end-start)/len(df), 3)} ms")

Time taken per message for CNN: 0.926 ms


In [12]:
print(classification_report(y_pred, df.target))

              precision    recall  f1-score   support

           0       0.74      0.96      0.84       386
           1       0.97      0.79      0.87       614

    accuracy                           0.86      1000
   macro avg       0.86      0.88      0.85      1000
weighted avg       0.88      0.86      0.86      1000

